In [ ]:

import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess_mjy as preprocess
# import preprocess as prep

# import SLR_model
import SLR_model_CNN_GRU
# import SLR_model_CL_GRU
import numpy as np



# model will output multiple(5) results/sec, how are we gonna handle it?


2.17.0


In [ ]:
load_size = 3000 # number of data to be loaded at once
epochs = 10
run_time= 5
# batch_size = 16
save_dir = "saves_CNN_GRU"
load_dir = "saves_CNN_GRU"
model = SLR_model_CNN_GRU.get_model()

In [20]:
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)

In [ ]:
# importlib.reload(SLR_model_CNN_GRU)

<module 'SLR_model_CNN_GRU' from 'd:\\Users\\jerry.DESKTOP-KQESKMB\\Desktop\\pyt\\sign\\git_folder\\BomNae-SLR\\SLR_model_CNN_GRU.py'>

In [3]:
cngru_model = SLR_model_CNN_GRU.reinit_model(run_eagerly=True)

In [4]:
# PER VIDEO TRAINING LOOP

end_file=preprocess.getoutputdir()

save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"

check_path = os.path.join(save_dir,'ckpt',ckpt_name)
hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=f'{check_path}.keras',
monitor='categorical_accuracy',
mode='max',
save_freq='epoch',
save_best_only=True)


start_person=1

for i in range(1,17):
    if start_person>i:
        continue
    for k in range(1,run_time+1):
        chkpt_elapsed = 0
        file_list = sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i))))
        for j in file_list: # every video
            print(j)
            l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
            with open(os.path.join('logs',ckpt_name+'.txt'), 'a') as logs:
                hist = None # declaration for scope
                logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {j}\n')  # 한 줄 쓰기
                ## ----------------------------------------------------
                ## CLGRU <-> CNNGRU 전환시 serialize 인자값 바꿔줄 것
                ## --------------------------------------
                # PER VIDEO TRAINING LOOP
                l_train, window_count = SLR_model_CNN_GRU.serialize(l_raw, stride=1, window_size=17, hop_length=10)
                r_train, window_count = SLR_model_CNN_GRU.serialize(r_raw, stride=1, window_size=17, hop_length=10)
                p_train, window_count, sample_weights = SLR_model_CNN_GRU.serialize(p_raw, stride=2, window_size=17, hop_length=10, loss_weights=loss_weights_raw, is_pose=True)
                x_train = (l_train, r_train, p_train)

                # y_train = np.repeat(y_raw, window_count)
                # print(y_raw.shape)
                y_train = SLR_model_CNN_GRU.encode_onehot2d(y_raw)
                y_train = tf.expand_dims(y_train, axis=0)
                # print('yt:', y_train.shape)
                # print("li:",x_train[0].shape)
                # print("ri:",x_train[1].shape)
                # print(x_train[2].shape)
                # print(sample_weights.shape)
                # dataset = SLR_model_CNN_GRU.convert_to_dataset(x_train, y_train, batch_size= 1)#, sample_weights= sample_weights)
                # hist = cngru_model.fit(x_train, y_train, batch_size=1, epochs=epochs)
                                
                if chkpt_elapsed < load_size and j is not file_list[-1]:
                    # train without checkpoint
                    # print(cngru_model.summary())
                    hist = cngru_model.fit(x_train, y_train, batch_size=1, epochs=epochs)
                    chkpt_elapsed += 1
                else:
                    # train with checkpoint every {load_size}th video
                    hist = cngru_model.fit(x_train, y_train, batch_size=1, epochs=epochs, callbacks=[model_checkpoint_callback])
                    chkpt_elapsed = 0   

                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)

0001.npz
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - categorical_accuracy: 0.0000e+00 - loss: 8.1983
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 5.0815
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 3.8091
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 1.9722
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 1.1890
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 0.4406
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 0.1122
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 0.0338
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 0.0119
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 1.0000 - loss: 0.0043
0002.npz
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step 

KeyboardInterrupt: 

In [ ]:
# LEGACY TRAINING LOOP WITH LOAD SIZE  

 # reload model file
end_file=preprocess.getoutputdir()

save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"

check_path = os.path.join(save_dir,'ckpt',ckpt_name)
hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=f'{check_path}.keras',
monitor='categorical_accuracy',
mode='max',
save_freq='epoch',
save_best_only=True)


start_person=1
start_count=1
current_word=" "

for i in range(1,17):
    l_raws=[]
    r_raws=[]
    p_raws=[]
    y_raws=[]
    loss_weights_raws=[]
    if start_person>i:
        continue
    for k in range(1,run_time+1):
        if start_count>k:
            continue
        elif start_count==k:
            start_count=0
        for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
            if len(l_raws)==0:
                current_word = j
            else:
                end_word = j
            l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
            l_raws.append(l_raw)
            r_raws.append(r_raw)
            p_raws.append(p_raw)
            y_raws.append(y_raw)
            loss_weights_raws.append(loss_weights_raw)
            # print(p_raw.shape)
            # break
            if len(l_raws)>=load_size:
                with open(os.path.join('logs',ckpt_name+'.txt'), 'a') as logs:
                    logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {current_word} ~ {end_word}\n')  # 한 줄 쓰기
                    l_train, each = SLR_model_GRU.serialize(l_raws)
                    r_train, each = SLR_model_GRU.serialize(r_raws)
                    p_train, each, sample_weights = SLR_model_GRU.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                    x_train = (l_train, r_train, p_train)
                    
                    y_train = np.repeat(y_raws, each)
                    y_train = SLR_model_GRU.encode_onehot2d(y_train)
                    
                    dataset = SLR_model_GRU.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                    hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])

                    with open(hist_path, 'w') as file:
                        json.dump(hist.history, file)
                    l_raws.clear()
                    r_raws.clear()
                    p_raws.clear()
                    y_raws.clear()
                    loss_weights_raws.clear()
            if len(l_raws)>0:
                l_train, each = SLR_model_GRU.serialize(l_raws)
                r_train, each = SLR_model_GRU.serialize(r_raws)
                p_train, each, sample_weights = SLR_model_GRU.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
                x_train = (l_train, r_train, p_train)
                
                y_train = np.repeat(y_raws, each)
                y_train = SLR_model_GRU.encode_onehot2d(y_train)
                
                
                dataset = SLR_model_GRU.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
                hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])

                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)



In [ ]:
# Configurations
load_size = 3000 # number of data to be loaded at once
epochs = 50
run_time = 2
batch_size = 16
save_dir = "saves"
load_dir = "saves"
# load_path = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints.keras"


In [ ]:
model = SLR_model.get_model()

In [ ]:
# # 세이브된 모델 로드
# load_path = os.join(load_dir, "check_00-00-00.keras")
# model = SLR_model.load_model(load_path)

In [ ]:
# # 안될때 (강제중지 + 초기화)
# tf.keras.backend.clear_session()

In [ ]:
with open(os.path.join("saves_GRU","hist","11-06-21-50epochs-2times.json"), 'r') as f:
    hist = json.load(f)

    plt.plot(range(len(hist['loss'])), hist['loss'])
    plt.scatter(range(len(hist['loss'])), hist['loss'])
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend(["loss"])
    plt.ylim((0,1))
    plt.show()

In [ ]:
# print(os.listdir(os.path.join(preprocess.getoutputdir(),str(15))))

In [ ]:
# model.metrics_names